In [ ]:
# standard libraries
# import os
import datetime as dt
import time

from typing import Dict, List, Optional

# external tools
from IPython.display import clear_output
import pandas as pd
import plotly.express as px
from rich import print
import speedtest


In [ ]:
output_file = "test.csv"
history_file = "history.csv"


In [ ]:
def convert_results_to_dataframe(results: Dict) -> pd.DataFrame:
    """
    Convert the dict results from a speedtest object into a dataframe

    :param results: dictionary of results and stats from a speedtest
    :type results: Dict
    :return: dataframe of results from selective fields
    """
    results_df = pd.DataFrame.from_dict(
        {
            'server_name': [results['server']['name']],
            'server_id': [results['server']['id']],
            'download': [round(results['download'], 0)],
            'upload': [round(results['upload'], 0)],
            'ping': [round(results['ping'], 1)],
            'server_url': [results['server']['url']],
            'timestamp': [results['timestamp']],
        },
    )
    
    return results_df


def run_speedtest(
    direction: Optional[List[str]] = None,
    threads: Optional[int] = None,
    server_list: Optional[List[int]] = None,
) -> speedtest.Speedtest:
    """
    Run a speedtest and return the results as a dataframe
    
    :param speedtest_obj: 
    :type speedtest_obj: speedtest.Speedtest
    :param direction: choose from 'up', 'down', or 'both'
    :type direction: str defaults to 'both'
    :param threads: number of threads to use
    :type threads: int defaults to None
    :param server_list 
    :type server_list: Optional[List[int]] defaults to None
    :rtype: speedtest.Speedtest
    """
    speedtest_obj = speedtest.Speedtest()
    direction = direction or ['down', 'up']
    servers = server_list or []

    print(f"Getting best server from {'all servers' if isinstance(servers, list) else servers}")
    speedtest_obj.get_servers(servers)
    speedtest_obj.get_best_server()
    
    for this_direction in direction:
        if this_direction not in ['down', 'up']:
            raise ValueError(f"direction must be down or up, not {this_direction}")
        if this_direction == "down":
            print("Running download test")
            speedtest_obj.download(threads=threads)

        if this_direction == "up":
            print("Running upload test")
            speedtest_obj.upload(threads=threads)

    return speedtest_obj


def show_updated_speed(history_df: pd.DataFrame) -> None:
    """
    Show the updated speed test results using Plotly Express
    
    :param history_df: dataframe of historical speed test results
    :type history_df: pd.DataFrame
    :return: None
    """
    
    download_df = history_df[['timestamp', 'download']].rename({'download': 'mbps'}, axis=1)
    download_df["type"] = "down"
    
    upload_df = history_df[['timestamp', 'upload']].rename({'upload': 'mbps'}, axis=1)
    upload_df["type"] = "up"
    
    plot_df = pd.concat([download_df, upload_df], axis=0)
    
    # convert from MBps to Mbps (2nd one is what everyone tells you you're getting)
    # plot_df.loc[:, 'mbps'] *= 8 / 1e6
    
    fig = px.line(plot_df, x='timestamp', y='mbps', color='type')
    fig.show()
    
    print(history_df[['download', 'upload']].describe())

    return None

In [ ]:
# If you want to test against a specific set of servers
# servers = [1234, 5678, 9012]
# None will automatically pick the best server from all available
# You can view this list by running s.get_servers() (see below)
servers = None

# If you want to use a single threaded test
# threads = 1
threads = None

history_df = pd.read_csv(f"data\{history_file}")

# delay between testing runs
wait = 0.5  # min

loops = 10

In [ ]:
for i in range(loops):
    start = time.time()
    print(f"{i+1}: {dt.datetime.now()}")

    s = run_speedtest(threads=threads, server_list=servers)
    data_df = convert_results_to_dataframe(s.results.dict())
    history_df = pd.concat([history_df, data_df], axis=0)

    history_df.to_csv(f"data\{history_file}", index=False)
   
    if i < loops:
        # s.results.share()  # generates a link to the results.  Doesn't seem to be working in Jupyter
        end = time.time()
        print(f"{i+1}/{loops} -- Waiting for {wait*60} seconds")
        time.sleep(wait * 60)
        clear_output(wait=False)

print("Done")

### Rechart

In [ ]:
show_updated_speed(history_df)

In [ ]:
history_df.head()

In [ ]:
s = speedtest.Speedtest()
s.get_servers()

In [ ]:
print(s.results)